# Restricted Boltzmann Machine

## Get dataset

In [ ]:
wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [2]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

print(movies)
# print(users)
# print(ratings)

         0                                   1                             2
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy
...    ...                                 ...                           ...
3878  3948             Meet the Parents (2000)                        Comedy
3879  3949          Requiem for a Dream (2000)                         Drama
3880  3950                    Tigerland (2000)                         Drama
3881  3951             Two Family House (2000)                         Drama
3882  3952               Contender, The (2000)                Drama|Thriller

[3883 rows x 3 columns]


## Preparing the training set and the test set


In [3]:
training_set = pd.read_csv('ml-1m/training_set.csv')
print(training_set)
training_set = np.array(training_set, dtype='int')
# print(training_set)
test_set = pd.read_csv('ml-1m/test_set.csv')
print(test_set)
test_set = np.array(test_set, dtype='int')
# print(test_set)


        User  Movie  Rating  Timestamp
0          1    661       3  978302109
1          1    914       3  978301968
2          1   3408       4  978300275
3          1   2355       5  978824291
4          1   1287       5  978302039
...      ...    ...     ...        ...
750116  6040   1091       1  956716541
750117  6040   1094       5  956704887
750118  6040    562       5  956704746
750119  6040   1096       4  956715648
750120  6040   1097       4  956715569

[750121 rows x 4 columns]
        User  Movie  Rating  Timestamp
0          1   1193       5  978300760
1          1   1197       3  978302268
2          1   2804       5  978300719
3          1    595       5  978824268
4          1    938       4  978301752
...      ...    ...     ...        ...
250083  6040   3735       4  960971654
250084  6040   2791       4  956715569
250085  6040    527       5  956704219
250086  6040   2003       1  956716294
250087  6040    535       4  964828734

[250088 rows x 4 columns]


## Getting the number of users and movies


In [4]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
print(f"number of users {nb_users}")
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(f"number of movies {nb_movies}")

number of users 6040
number of movies 3952


## Converting the data into an array with users in lines and movies in columns


In [5]:
# Create list of lists with length equal to number of movies
def convert(data):
	new_data = []
	for id_users in range(1, nb_users + 1):
		id_movies = data[:,1][data[:,0]== id_users]
		id_ratings = data[:,2][data[:,0] == id_users]
		ratings = np.zeros(nb_movies)
		ratings[id_movies - 1] = id_ratings
		new_data.append(list(ratings))
	return new_data


training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [6]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)


In [7]:
# Movies that have not been rated are set to -1
training_set[training_set == 0] = -1 
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1 
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Creating the architecture of the Neural Network


In [8]:
class RBM():
	def __init__(self, nv, nh): # nv - visible nodes, nh - hidden nodes
		self.W = torch.randn(nh, nv) # W - Weights
		self.a = torch.randn(1, nh) # a - Biass for hidden nodes
		self.b = torch.randn(1, nv) # b - Biass for visible nodes
	def sample_h(self, x): # calculate probability of a hidden node activating based on visible node
		wx = torch.mm(x, self.W.t())
		activation = wx + self.a.expand_as(wx)
		p_h_given_v = torch.sigmoid(activation)
		return p_h_given_v, torch.bernoulli(p_h_given_v)
	def sample_v(self, y): # calculate probability of a visible node activating based on hidden node
		wy = torch.mm(y, self.W)
		activation = wy + self.b.expand_as(wy)
		p_v_given_h = torch.sigmoid(activation)
		return p_v_given_h, torch.bernoulli(p_v_given_h)
	def train(self, v0, vk, ph0, phk):
		self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
		self.b += torch.sum((v0 - vk), 0)
		self.a += torch.sum((ph0 - phk), 0)


rbm = RBM(nv=len(training_set[0]), nh=100)

## Training the RBM


In [10]:
batch_size = 100
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
	train_loss = 0
	s = 0.
	for id_user in range(0, nb_users - batch_size, batch_size):
		vk = training_set[id_user:id_user + batch_size]
		v0 = training_set[id_user:id_user + batch_size]
		ph0,_ = rbm.sample_h(v0)
		for k in range(10):
			_,hk = rbm.sample_h(vk)
			_,vk = rbm.sample_v(hk)
			vk[v0<0] = v0[v0<0]
		phk,_ = rbm.sample_h(vk)
		rbm.train(v0, vk, ph0, phk)
		train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
		s += 1.
	print(f" Epoch: {epoch}, loss: {train_loss/s}")

 Epoch: 1, loss: 0.24925334751605988
 Epoch: 2, loss: 0.22773629426956177
 Epoch: 3, loss: 0.22858811914920807
 Epoch: 4, loss: 0.2285548746585846
 Epoch: 5, loss: 0.22808106243610382
 Epoch: 6, loss: 0.22710739076137543
 Epoch: 7, loss: 0.22793985903263092
 Epoch: 8, loss: 0.22773613035678864
 Epoch: 9, loss: 0.22777631878852844
 Epoch: 10, loss: 0.22753086686134338


## Testing the RBM


In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
	v = training_set[id_user:id_user+1]
	vt = test_set[id_user:id_user+1]
	ph0,_ = rbm.sample_h(v0)
	if len(vt[vt>=0]) > 0:
		_,h = rbm.sample_h(v)
		_,v = rbm.sample_v(h)
		test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
		s += 1.
print(f" test loss: {test_loss/s} ")

 test loss: 0.2114362120628357 


## We predicted the correct rating (0 or 1) roughly 4/5 times 

# More advanced recommendation system can be found in AE/AutoEncoder